## Import Statements

In [ ]:
import gensim.downloader as api
import numpy as np
import re
import csv
import pandas as pd
import pprint
import string
import nltk
import sys

from IPython.display import HTML
from nltk.corpus import wordnet 
from sklearn.manifold import TSNE
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
sys.path.insert(0, pathToDataScripts)
from cleanDataset import tokenize_words 

pathToDatasets = '../datasets/'
pathToDataScripts = '../datasets/scripts/'
filePath = '../datasets/GoogleNews-vectors-negative300.bin'

## Downloading binaries and models


In [ ]:
word_vectors = api.load("glove-wiki-gigaword-100")
nltk.download('vader_lexicon')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

## Global Variables and Global Objects

In [ ]:
senty = SentimentIntensityAnalyzer()
vocabulary = word_vectors.vocab;


NUMBER_OF_ALTERNATIVES = 7
TWEET_START = 50
NUM_OF_TWEETS = 25

punctuation = r"\"#$%&'()+-/:;<=>?@[\]*^_`{|}~"

## Class for Sentences


In [ ]:
class Sentence:
    ogSentence = ""
    ogSentiment = 0;
    indexOfWords = [];
    htmlSentence = "";
    sentencesToSentiments = {}
    
    def __init__(self, sentence, sentiment):
        self.ogSentence = sentence;
        self.ogSentiment = sentiment;
        self.htmlSentence = sentence;
        self.indexOfWords = [];
        self.sentencesToSentiments[self.ogSentence] = self.ogSentiment;

    

## Utility Code

In [ ]:
def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

def cleanAndTokenizeText(text):
    text = text.lower();
    newString = ""
    for char in text:
        if char not in punctuation:
            newString += char
    text = word_tokenize(newString)
    return text;

def getPOSTags(tweet):
    tags = nltk.pos_tag(tweet)
    return tags;

def getAntonymsOfWords(word):
    if(word not in vocabulary):
        return []
    setOfAntonyms = set()
    for syn in wordnet.synsets(word):
        for l in syn.lemmas():
            anton = l.antonyms()
            if(anton!=[]):
                setOfAntonyms.add(anton[0].name())
    return list(setOfAntonyms)

def listReplacements(word):
    if(word not in vocabulary):
        return []
    possibleReplacements = [word[0] for word in word_vectors.most_similar(word,topn=NUMBER_OF_ALTERNATIVES)]
    antonyms = getAntonymsOfWords(word)
    if(antonyms != []):
        possibleReplacements.extend(antonyms)
        return possibleReplacements

    
def posApprovedReplacements(alternativeWords, userTokens, indexOfToken):
    if(alternativeWords = []):
        print("---- NO ALTERNATIVE WORDS! ----");
        return []
    tempTokens = userTokens[:]
    POSTokens = getPOSTokens(tempTokens)
    validWords = []
    
    mainTag = POSTokens[indexOfToken][1]
    mainWord = userTokens[indexOfToken]
    